In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[7]:

In [ ]:
get_ipython().magic('pylab inline')

Stochastic Simulation<br>
================================<br>
<br>
This tutorial shows the basics of running a simulation driven by noise (ie, using <br>
a stochastic integration scheme). It discusses the choices around specifying the <br>
driving stochastic process, and then performs a simple simulation to show the <br>
results.<br>
<br>
Here we'll use a region level simulation, but the considerations for surface<br>
simulations are the same.

We'll start as usual by importing a bunch of tools:

In[8]:

In [ ]:
from tvb.simulator.lab import *
LOG = get_logger('demo')

and then initialising our standard region level <br>
simulation: with a Generic2dOscillator model; the default 74 region Connectivity, with a<br>
conduction speed set to 4ms$^{-1}$; a Linear coupling function with a scaling factor of <br>
0.0152; and two Monitors, one raw, ie returning all the simulated data, and the other a<br>
TemporalAverage, which we'll set to record at 2000Hz and by default only returns the <br>
state-variables specified in the Model's variables_of_interest attribute.

In[9]:

nitialise a Model, Coupling, and Connectivity.

In [ ]:
oscillator = models.Generic2dOscillator()
white_matter = connectivity.Connectivity(load_default=True)
white_matter.speed = numpy.array([4.0])

In [ ]:
white_matter_coupling = coupling.Linear(a=0.0152)

nitialise some Monitors with period in physical time

In [ ]:
mon_raw = monitors.Raw()
mon_tav = monitors.TemporalAverage(period=2**-1)

undle them

In [ ]:
what_to_watch = (mon_raw, mon_tav)

A Stochastic (Noisy) Integration Scheme<br>
----------------------------------------<br>
<br>
Noise enters through the integration scheme, here we'll define a simple constant level <br>
of noise that enters all nodes and all state variables, however, the noise is <br>
configurable on a per node and per state variable level, and as such the noise can be <br>
reconfigured to, for example, only enter appropriate state variables of certain thalamic <br>
nodes, thus emulating a very crude model of brain stem input to the brain.<br>
<br>
We can choose between Additive and Multiplicative Noise functions when defining<br>
the way noise enters our simulation. In the case of Multiplicative Noise the function<br>
depends on the state of the system.<br>
<br>
The Noise functions are fed by a NoiseStream, that is, a random process generated by<br>
a pseudo-random number generator. The random processes used have Gaussian amplitude <br>
and can potentially be given a temporal correlation. The random process is defined <br>
using two parameters plus the state of the NoiseStream. The two parameters are: nsig,<br>
defining the standard deviation of the noise amplitude; and ntau which defines the <br>
correlation time of the noise source, with ntau = 0 corresponding to white noise and <br>
any value greater than zero producing coloured noise. The state of the NoiseStream<br>
can be specified using a random seed, which is just a number, by default the random <br>
seed is 42.<br>
<br>
Here, we'll specify a small amplitude white noise and pass it into our stochastic <br>
integrator. As a general rule of thumb, the noise amplitude, when being applied to <br>
all nodes and all state-variables, should be much less than 1% of the expected <br>
state-variable range if you don't want the noise to dominate the intrinsic dynamics<br>
of the system.

In[10]:

nitialise an Integrator

In [ ]:
hiss = noise.Additive(nsig = numpy.array([2**-10,]))
heunint = integrators.HeunStochastic(dt=2**-4, noise=hiss)

We can get a quick idea of what the noise amplitude that we've set looks like in comparison<br>
to our chosen Model using the PhasePlaneInteractive plotting tool. For more details about using<br>
PhasePlaneInteractive, see "Tutorial: Exploring A Model". This can be achieved with: <br>
<br>
    from tvb.simulator.plot.phase_plane_interactive import PhasePlaneInteractive<br>
    <br>
    #Create and launch an interactive phase plane plot<br>
    ppi_fig = PhasePlaneInteractive(model=oscillator, integrator=heunint)<br>
    ppi_fig.show()<br>
<br>
**NOTE:** *Modifying parameters in PhasePlaneInteractive will modify the corresponding <br>
parameters of the Model and Integrator passed into it, so be careful...*

Simulate<br>
--------<br>
<br>
We can now construct a Simulator including our stochastic integrator and run a simulation

In[11]:

nitialise a Simulator -- Model, Connectivity, Integrator, and Monitors.

In [ ]:
sim = simulator.Simulator(model = oscillator, 
                          connectivity = white_matter,
                          coupling = white_matter_coupling, 
                          integrator = heunint, 
                          monitors = what_to_watch)

In [ ]:
sim.configure()

erform the simulation

In [ ]:
raw_data = []
raw_time = []
tavg_data = []
tavg_time = []
for raw, tavg in sim(simulation_length=2**9):
    if not raw is None:
        raw_time.append(raw[0])
        raw_data.append(raw[1])
    
    if not tavg is None:
        tavg_time.append(tavg[0])
        tavg_data.append(tavg[1])

Squigily Lines...<br>
------------------<br>
<br>
And take a look at what we've done

In[12]:

ake the lists numpy.arrays for easier use.

In [ ]:
RAW = numpy.array(raw_data)
TAVG = numpy.array(tavg_data)

lot raw time series

In [ ]:
figure(1)
plot(raw_time, RAW[:, 0, :, 0])
title("Raw -- State variable 0")

In [ ]:
figure(2)
plot(raw_time, RAW[:, 1, :, 0])
title("Raw -- State variable 1")

lot temporally averaged time series

In [ ]:
figure(3)
plot(tavg_time, TAVG[:, 0, :, 0])
title("Temporal average")

how them

In [ ]:
show()

The large amplitude oscillations at the start are, of course, our usual large<br>
transient resulting from our being too lazy to properly set initial conditions.<br>
But after that we see a small amplitude noise driven exploration of the small<br>
piece of phase-space surrounding the fixed point produced by the default parameters<br>
for Generic2dOscillator.

That's All Folks... so, what next?<br>
----------------------------------<br>
<br>
That's all for this tutorial, the main new thing was a bit of discussion and an example<br>
of running a stochastic simulation.<br>
<br>
You might now want to take a look at Tutorial: blah, where we craft a detailed<br>
noise source designed to have a specific physical meaning.